In [1]:
import torch
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.get_device_name()}")

Using device: cuda
Current GPU: NVIDIA GeForce RTX 3060


In [4]:
import pandas as pd
import ast
import pickle
import torch
from torch import nn, optim
from sklearn.feature_extraction.text import CountVectorizer

# 칵테일 추천 모델 클래스 정의
class CocktailRecommendationModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CocktailRecommendationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 입력층 -> 은닉층
        self.fc2 = nn.Linear(hidden_size, input_size)  # 은닉층 -> 출력층
        self.relu = nn.ReLU()  # 활성화 함수

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 데이터 전처리 함수
def clean_ingredients(ingredients_str):
    try:
        ingredients_list = ast.literal_eval(ingredients_str)
        return [ingredient.strip().lower().replace(' ', '') for ingredient in ingredients_list]
    except Exception:
        return []

def preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['cleaned_ingredients'] = data['ingredients'].apply(clean_ingredients)
    data['ingredient_text'] = data['cleaned_ingredients'].apply(lambda x: ' '.join(x))
    vectorizer = CountVectorizer()
    ingredient_vectors = vectorizer.fit_transform(data['ingredient_text'])
    return data, vectorizer, ingredient_vectors

# 학습 함수
def train_model(ingredient_vectors, save_model_path, save_vectorizer_path, hidden_size=128, epochs=100, learning_rate=0.001):
    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 모델 초기화
    input_size = ingredient_vectors.shape[1]
    model = CocktailRecommendationModel(input_size, hidden_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # 데이터 텐서로 변환
    X = torch.tensor(ingredient_vectors.toarray(), dtype=torch.float32).to(device)

    # 학습 루프
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, X)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

    # 모델 저장
    torch.save(model.state_dict(), save_model_path)
    print(f"Model saved to {save_model_path}")

    # Vectorizer 저장
    with open(save_vectorizer_path, "wb") as f:
        pickle.dump(vectorizer, f)
    print(f"Vectorizer saved to {save_vectorizer_path}")

# 학습 실행
if __name__ == "__main__":
    dataset_path = r"D:\project\OSS_Project\AI\recommend\data\final_cocktails.csv"
    model_save_path = r"D:\project\OSS_Project\AI\recommend\cocktail_recommender.pt"

    vectorizer_save_path = "vectorizer.pkl"

    # 데이터 전처리
    data, vectorizer, ingredient_vectors = preprocess_data(dataset_path)

    # 모델 학습 및 저장
    train_model(ingredient_vectors, model_save_path, vectorizer_save_path)


Using device: cuda
Epoch [10/100], Loss: 0.0227
Epoch [20/100], Loss: 0.0196
Epoch [30/100], Loss: 0.0171
Epoch [40/100], Loss: 0.0146
Epoch [50/100], Loss: 0.0120
Epoch [60/100], Loss: 0.0097
Epoch [70/100], Loss: 0.0077
Epoch [80/100], Loss: 0.0061
Epoch [90/100], Loss: 0.0049
Epoch [100/100], Loss: 0.0040
Model saved to cocktail_recommender.pt
Recommended Cocktails:
A1: 0.00


C:\Users\user\AppData\Local\Temp\ipykernel_21144\3150378286.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path))
